# Clean Architecture on Python

# Advantages
 - Independent of Frameworks
 - Testable
 - Independent of UI
 - Independent of Database
 - Independent of any external agency

Each of these architectures produce systems that are:
 - Independent of Frameworks. The architecture does not depend on the existence of some library of feature laden software. This allows you to use such frameworks as tools, rather than having to cram your system into their limited constraints.
 - Testable. The business rules can be tested without the UI, Database, Web Server, or any other external element.
 - Independent of UI. The UI can change easily, without changing the rest of the system. A Web UI could be replaced with a console UI, for example, without changing the business rules.
 - Independent of Database. You can swap out Oracle or SQL Server, for Mongo, BigTable, CouchDB, or something else. Your business rules are not bound to the database.
 - Independent of any external agency. In fact your business rules simply don’t know anything at all about the outside world.

![Clean Architecture](CleanArchitecture.jpg)

In [5]:
# Don't do this!
import json
import requests
from urllib.parse import urlencode

def convert(base, dest, amount, date_obj=None):
    if date_obj is None:
        date_str = 'latest'
    else:
        date_str = date_obj.strftime('%F')
    payload = {'base': base, 'rtype': 'fpy', 'symbols': dest}
    url = f'https://api.ratesapi.io/api/{date_str}'
    response = requests.get(url, params=payload)  # I/O
    json_data = response.json()  # I/O
    rates = json_data['rates']
    try:
        rate = rates[dest]
    except KeyError:
        raise NotImplementedError(f'{dest} not supported')
    converted_amount = rate * amount
    return converted_amount

In [5]:
# Let's hide the I/O
import json
import requests
from urllib.parse import urlencode

def call_json_api(url, payload):
    response = requests.get(url, params=payload)  # I/O
    return response.json()  # I/O

def convert(base, dest, amount, date_obj=None):
    if date_obj is None:
        date_str = 'latest'
    else:
        date_str = date_obj.strftime('%F')
    payload = {'base': base, 'rtype': 'fpy', 'symbols': dest}
    url = f'https://api.ratesapi.io/api/{date_str}'
    json_data = call_json_api(url, payload)
    rates = json_data['rates']
    try:
        rate = rates[dest]
    except KeyError:
        raise NotImplementedError(f'{dest} not supported')
    converted_amount = rate * amount
    return converted_amount

In [7]:
# Decouple it instead
import json
import requests
from urllib.parse import urlencode

def build_url(date_obj):
    if date_obj is None:
        date_str = 'latest'
    else:
        date_str = date_obj.strftime('%F')
    return f'https://api.ratesapi.io/api/{date_str}'

def construct_payload(base, dest):
    return {'base': base, 'rtype': 'fpy', 'symbols': dest}

def get_converted_amount(json_data):
    rates = json_data['rates']
    try:
        rate = rates[dest]
    except KeyError:
        raise NotImplementedError(f'{dest} not supported')
    return rate * amount


In [7]:
def convert(base, dest, amount, date_obj=None):
    date_str = get_date_str(date_obj)
    payload = construct_payload(base, dest)
    response = requests.get(url, params=payload)  # I/O
    json_data = response.json()  # I/O
    return get_converted_amount(json_data)

![request-report-1](request-report-1.png)

### Original Code
 - `request_report` -- all in one function
     - Transactions
         - Wait for Report from MWS
         - Retrieve
         - Parse
     - Adjustments
         - Request
         - Wait
         - Retrieve
         - Parse
     - Reimbursements
         - Request
         - Wait
         - Retrieve
     - Collate Adjustments & Reimbursements
         - Sort per SKU
         - Deduct Reimbursements from Adjustments
         - List SKUs that need reimbursements
     - For each SKU, get prices from Transactions
     

### First rewrite
- `ReportBaseClass`
    - request
    - wait_and_retrieve
    - parse
- `request_report`
    - call `wait_and_retrieve` and `parse` from `TransactionReport`
    - call `request`, `wait_and_retrieve` and `parse` from `ReimbursementReport` and `AdjustmentReport`
    - collate & get prices like before

References:
 - https://blog.cleancoder.com/uncle-bob/2012/08/13/the-clean-architecture.html
 - https://github.com/python-leap/book
 - https://www.blog.pythonlibrary.org/2018/09/25/creating-presentations-with-jupyter-notebook/